In [1]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.4 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('exam_dataset_50k.csv')

/tmp/ipython-input-2939/3095725691.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('exam_dataset_50k.csv')


In [4]:
# Install necessary libraries
!pip install xgboost scikit-learn pandas matplotlib seaborn

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
import xgboost as xgb

In [6]:
# Load the dataset
df = pd.read_csv("/content/exam_dataset_50k.csv", low_memory=False)
print(f"Loaded: {df.shape[0]:,} rows x {df.shape[1]} columns\n")

Loaded: 50,000 rows x 68 columns



In [7]:
# Available when a question is written -- before any student sees it
TEXT_FEATURES = [
    "text_length", "word_count", "sentence_count", "avg_word_length",
    "latex_command_count", "has_latex", "latex_density",
    "math_operator_count", "number_count", "vocab_richness",
    "text_complexity_score",
    "answer_a_length", "answer_b_length", "answer_c_length", "answer_d_length",
    "avg_answer_length", "answer_length_variance",
    "has_advanced_terms", "has_algebra_terms",
    "has_geometry_terms", "has_stats_terms",
    "num_misconceptions", "has_misconception",
    "subject_difficulty_tier", "construct_frequency",
]

# Available after the first exam sitting, but never directly encoding p-value
POST_ADMIN_FEATURES = TEXT_FEATURES + [
    "avg_response_time_sec",
    "std_response_time_sec",
    "discrimination_index",
    "point_biserial_corr",
    "irt_a_param",
]

In [8]:
# Define regression and classification targets
REG_TARGET = "difficulty_p_value"    # continuous
CLF_TARGET = "difficulty_label"      # Easy / Medium / Hard

y_reg = df[REG_TARGET].astype(float)

le = LabelEncoder()
y_clf = le.fit_transform(df[CLF_TARGET])
CLASS_NAMES = le.classes_
N_CLASSES   = len(CLASS_NAMES)

print("Classes :", list(CLASS_NAMES))
print("Counts  :", {c: int((y_clf==i).sum()) for i,c in enumerate(CLASS_NAMES)})

Classes : ['Easy', 'Hard', 'Medium']
Counts  : {'Easy': 39189, 'Hard': 1740, 'Medium': 9071}


In [9]:
# Shared XGBoost parameters for regression and classification
XGB_PARAMS_REG = dict(
    n_estimators     = 400,
    learning_rate    = 0.05,
    max_depth        = 6,
    subsample        = 0.8,
    colsample_bytree = 0.8,
    reg_alpha        = 0.1,
    reg_lambda       = 1.0,
    random_state     = 42,
    n_jobs           = -1,
    verbosity        = 0,
)

XGB_PARAMS_CLF = dict(
    **XGB_PARAMS_REG,
    objective         = "multi:softprob",
    num_class         = N_CLASSES,
    use_label_encoder = False,
    eval_metric       = "mlogloss",
)

results = {}   # Dictionary to store all metric dicts

## TEXT-ONLY FEATURES

In [13]:
Xa  = df[TEXT_FEATURES]
yar = df[REG_TARGET]
yac = y_clf

Xa_tr, Xa_te, yar_tr, yar_te, yac_tr, yac_te = train_test_split(
    Xa, yar, yac, test_size=0.2, random_state=42, stratify=yac
)

In [14]:
# A Regressor
print("\n  [A-Reg] XGBoost Regressor ...")
print("\n  [A-Reg] XGBoost Regressor ...")

a_reg = xgb.XGBRegressor(**XGB_PARAMS_REG)
a_reg.fit(Xa_tr, yar_tr, eval_set=[(Xa_te, yar_te)], verbose=False)

a_reg_pred = a_reg.predict(Xa_te)

a_mae  = mean_absolute_error(yar_te, a_reg_pred)
a_rmse = np.sqrt(mean_squared_error(yar_te, a_reg_pred))
a_r2   = r2_score(yar_te, a_reg_pred)

print(f"    MAE={a_mae:.4f}  RMSE={a_rmse:.4f}  R2={a_r2:.4f}")


  [A-Reg] XGBoost Regressor ...

  [A-Reg] XGBoost Regressor ...
    MAE=0.0772  RMSE=0.0963  R2=0.5690


In [15]:
# A Classifier
print("  [A-Clf] XGBoost Classifier ...")
a_clf = xgb.XGBClassifier(**XGB_PARAMS_CLF)
a_clf.fit(Xa_tr, yac_tr, eval_set=[(Xa_te, yac_te)], verbose=False)
a_clf_pred  = a_clf.predict(Xa_te)
a_clf_proba = a_clf.predict_proba(Xa_te)

a_acc  = accuracy_score(yac_te, a_clf_pred)
a_prec = precision_score(yac_te, a_clf_pred, average="weighted", zero_division=0)
a_rec  = recall_score(yac_te, a_clf_pred, average="weighted", zero_division=0)
a_f1   = f1_score(yac_te, a_clf_pred, average="weighted", zero_division=0)
print(f"    Acc={a_acc:.4f}  Prec={a_prec:.4f}  Rec={a_rec:.4f}  F1={a_f1:.4f}")

  [A-Clf] XGBoost Classifier ...
    Acc=0.8331  Prec=0.8049  Rec=0.8331  F1=0.8137


In [16]:
# A Cross-Validation
print("  [A-CV]  5-Fold CV ...")
a_cv = cross_val_score(
    xgb.XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=6,
                      use_label_encoder=False, eval_metric="mlogloss",
                      objective="multi:softprob", num_class=N_CLASSES,
                      random_state=42, n_jobs=-1, verbosity=0),
    Xa, y_clf, cv=StratifiedKFold(5, shuffle=True, random_state=42),
    scoring="accuracy", n_jobs=-1
)
print(f"    CV Accuracy: {a_cv.mean():.4f} +/- {a_cv.std():.4f}")

  [A-CV]  5-Fold CV ...
    CV Accuracy: 0.8374 +/- 0.0027


In [17]:
results["A"] = dict(
    label="Text-Only", n_feat=len(TEXT_FEATURES),
    MAE=a_mae, RMSE=a_rmse, R2=a_r2,
    Acc=a_acc, Prec=a_prec, Rec=a_rec, F1=a_f1,
    CV=a_cv.mean(), CV_std=a_cv.std(),
    reg_pred=a_reg_pred, clf_pred=a_clf_pred, clf_proba=a_clf_proba,
    yr_te=yar_te, yc_te=yac_te,
    feat_imp_reg=a_reg.feature_importances_,
    feat_imp_clf=a_clf.feature_importances_,
    feat_names=np.array(TEXT_FEATURES),
)

print("\n  Full Classification Report (A):")
print(classification_report(yac_te, a_clf_pred, target_names=CLASS_NAMES))


  Full Classification Report (A):
              precision    recall  f1-score   support

        Easy       0.88      0.96      0.92      7838
        Hard       0.32      0.11      0.16       348
      Medium       0.57      0.42      0.48      1814

    accuracy                           0.83     10000
   macro avg       0.59      0.49      0.52     10000
weighted avg       0.80      0.83      0.81     10000



In [ ]:
import joblib

a_reg.save_model('xgb_reg_model_A.json')
a_clf.save_model('xgb_clf_model_A.json')

# Save the label encoder too
joblib.dump(le, 'xgb_text_model.pkl')

# Download the files
from google.colab import files
files.download('xgb_reg_model_A.json')
files.download('xgb_clf_model_A.json')
files.download('xgb_text_model.pkl')